In [2]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

In [3]:
#data needed to be catagorical
dataset = [['milk', 'onion', 'nutmag', 'kidney beans', 'eggs', 'yogurt'],
           ['dill', 'onion', 'nutmag', 'kidney beans', 'eggs', 'yogurt' ],
           ['milk','apple', 'kidney beans', 'eggs'],
           ['milk', 'unicorn', 'corn', 'kidney beans', 'yogurt'],
           ['corn', 'onion', 'onion', 'kidney beans', 'ice creams', 'eggs']]

In [5]:
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns = te.columns_)
print(df.head())

   apple   corn   dill   eggs  ice creams  kidney beans   milk  nutmag  onion  \
0  False  False  False   True       False          True   True    True   True   
1  False  False   True   True       False          True  False    True   True   
2   True  False  False   True       False          True   True   False  False   
3  False   True  False  False       False          True   True   False  False   
4  False   True  False   True        True          True  False   False   True   

   unicorn  yogurt  
0    False    True  
1    False    True  
2    False   False  
3     True    True  
4    False   False  


In [6]:
frequent_itemset = apriori(df, min_support=0.6, use_colnames=True)
print(frequent_itemset)

    support                     itemsets
0       0.8                       (eggs)
1       1.0               (kidney beans)
2       0.6                       (milk)
3       0.6                      (onion)
4       0.6                     (yogurt)
5       0.8         (eggs, kidney beans)
6       0.6                (onion, eggs)
7       0.6         (kidney beans, milk)
8       0.6        (onion, kidney beans)
9       0.6       (kidney beans, yogurt)
10      0.6  (onion, eggs, kidney beans)


In [ ]:
#adding the length of every basket to the dataset
frequent_itemset['length'] = frequent_itemset['itemsets'].apply(lambda x : len(x))
print(frequent_itemset)

    support                     itemsets  length
0       0.8                       (eggs)       1
1       1.0               (kidney beans)       1
2       0.6                       (milk)       1
3       0.6                      (onion)       1
4       0.6                     (yogurt)       1
5       0.8         (eggs, kidney beans)       2
6       0.6                (onion, eggs)       2
7       0.6         (kidney beans, milk)       2
8       0.6        (onion, kidney beans)       2
9       0.6       (kidney beans, yogurt)       2
10      0.6  (onion, eggs, kidney beans)       3


In [8]:
#finging a basket with the length of 3 and the support of more than 80 percent
frequent_itemset[ (frequent_itemset['length']>=2)&
                  (frequent_itemset['support']>=0.8)]

,support,itemsets,length
5,0.8,"(eggs, kidney beans)",2


In [11]:
#finding a basket that envolves onions and eggs
frequent_itemset[frequent_itemset['itemsets']== {'onion', 'eggs'}]

,support,itemsets,length
6,0.6,"(onion, eggs)",2


In [12]:
#sparsing the dataset
oht_ary = te.fit(dataset).transform(dataset, sparse = True)
sparse_df = pd.DataFrame.sparse.from_spmatrix(oht_ary, columns=te.columns_)
print(sparse_df)


   apple  corn  dill  eggs  ice creams  kidney beans  milk  nutmag  onion  \
0      0     0     0     1           0          True     1       1      1   
1      0     0     1     1           0          True     0       1      1   
2      1     0     0     1           0          True     1       0      0   
3      0     1     0     0           0          True     1       0      0   
4      0     1     0     1           1          True     0       0      1   

   unicorn  yogurt  
0        0       1  
1        0       1  
2        0       0  
3        1       1  
4        0       0  


In [13]:
apriori(sparse_df, min_support=0.6, use_colnames=True, verbose=1) #verbose is to give us info about what is processing rn

Processing 21 combinations | Sampling itemset size 3


,support,itemsets
0,0.8,(eggs)
1,1.0,(kidney beans)
2,0.6,(milk)
3,0.6,(onion)
4,0.6,(yogurt)
5,0.8,"(eggs, kidney beans)"
6,0.6,"(onion, eggs)"
7,0.6,"(kidney beans, milk)"
8,0.6,"(onion, kidney beans)"
9,0.6,"(kidney beans, yogurt)"


In [ ]:
from mlxtend.frequent_patterns import association_rules
myrules = association_rules(frequent_itemset, metric='confidence', min_threshold= 0.7) #min threshold is 
#antecedents is the things we bought and consequents is the items that are possible to be bought with them with the probability of the confidence 
myrules.head()

c:\ProgramData\Anaconda3\lib\site-packages\mlxtend\frequent_patterns\association_rules.py:186: RuntimeWarning: invalid value encountered in divide
  cert_metric = np.where(certainty_denom == 0, 0, certainty_num / certainty_denom)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(eggs),(kidney beans),0.8,1.0,0.8,1.00,1.00,1.0,0.00,inf,0.0,0.80,0.000,0.900
1,(kidney beans),(eggs),1.0,0.8,0.8,0.80,1.00,1.0,0.00,1.0,0.0,0.80,0.000,0.900
2,(onion),(eggs),0.6,0.8,0.6,1.00,1.25,1.0,0.12,inf,0.5,0.75,1.000,0.875
3,(eggs),(onion),0.8,0.6,0.6,0.75,1.25,1.0,0.12,1.6,1.0,0.75,0.375,0.875
4,(milk),(kidney beans),0.6,1.0,0.6,1.00,1.00,1.0,0.00,inf,0.0,0.60,0.000,0.800


In [ ]:
#for classification we should use the class as consequence
import numpy as np
#so this code classifies all the consequents that has eggs
myrules.iloc[np.array(myrules['consequents']=={'eggs'}),:]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
1,(kidney beans),(eggs),1.0,0.8,0.8,0.8,1.00,1.0,0.00,1.0,0.0,0.80,0.0,0.900
2,(onion),(eggs),0.6,0.8,0.6,1.0,1.25,1.0,0.12,inf,0.5,0.75,1.0,0.875
8,"(onion, kidney beans)",(eggs),0.6,0.8,0.6,1.0,1.25,1.0,0.12,inf,0.5,0.75,1.0,0.875


In [ ]:
from mlxtend.frequent_patterns import fpgrowth
#second way for making this dataset 
fpgrowth(df, min_support =0.6, use_colnames=True)

,support,itemsets
0,1.0,(kidney beans)
1,0.8,(eggs)
2,0.6,(yogurt)
3,0.6,(onion)
4,0.6,(milk)
5,0.8,"(eggs, kidney beans)"
6,0.6,"(kidney beans, yogurt)"
7,0.6,"(onion, eggs)"
8,0.6,"(onion, kidney beans)"
9,0.6,"(onion, eggs, kidney beans)"


In [ ]:
from mlxtend.frequent_patterns import fpgrowth
#second way for making this dataset 
fpgrowth(df, min_support =0.6, use_colnames=True)

,support,itemsets
0,1.0,(kidney beans)
1,0.8,(eggs)
2,0.6,(yogurt)
3,0.6,(onion)
4,0.6,(milk)
5,0.8,"(eggs, kidney beans)"
6,0.6,"(kidney beans, yogurt)"
7,0.6,"(onion, eggs)"
8,0.6,"(onion, kidney beans)"
9,0.6,"(onion, eggs, kidney beans)"
